In [7]:
from scoring import *

In [8]:
df_returns = pd.read_pickle(go_up(1) +
                                "/saved_data/ReturnsDataHuge.pkl")[:4030]
lookback_for_factors = 252
n_factor = 15
n_stocks = df_returns.shape[1]
trading_days = df_returns.shape[0] - lookback_for_factors
sp500cost = pd.read_pickle(
        go_up(1) + '/saved_data/SP500histcost_matched.pkl')
Q = np.zeros(shape=(trading_days, n_factor, n_stocks))
i=int(np.random.uniform(df_returns.shape[0]))

In [9]:
oneyear = sp500cost.iloc[i:lookback_for_factors + i]
period = df_returns[i:lookback_for_factors + i]

In [10]:
oneyearticks = []
alloneyearticks = np.unique(oneyear['Tickers'])
for l_ticks in alloneyearticks:
    for tick in l_ticks:
        oneyearticks.append(tick)
oneyearticks = list(set(oneyearticks))

In [11]:
period = period[period.columns.intersection(oneyearticks)]
period = period.dropna(axis=1)
period = period.loc[:, (period != 0).any(axis=0)]

In [12]:
period

,NOV,QCOM,USB,CTAS,SLM,TMO,ADSK,MAT,CMVT,PNC,...,AEP,DRI,GWW,COP,APD,BC,MTG,HAL,CINF,SOV
Date,,,,,,,,,,,,,,,,,,,,,
2004-12-23,-0.034841,-0.024916,-0.006355,-0.010519,-0.007836,-0.012378,-0.001333,-0.001045,-0.002072,0.000000,...,-0.008312,-0.021842,-0.001070,-0.012622,-0.009030,-0.021594,-0.005395,-0.023618,-0.003419,-0.023526
2004-12-27,0.012227,0.001611,0.008315,0.009013,0.006017,0.000989,0.029357,0.012552,0.009558,0.012033,...,0.003468,0.029029,0.013164,0.005811,0.009972,0.017946,0.020525,0.011065,0.008920,-0.011564
2004-12-28,0.019845,-0.015399,-0.011735,0.007329,0.000000,-0.000988,-0.007000,0.004649,-0.008228,-0.004546,...,-0.003456,-0.001808,-0.002719,0.008781,-0.001532,0.004661,-0.004597,0.009672,0.002494,0.004875
2004-12-29,-0.004512,-0.003501,-0.000321,-0.001364,0.003364,-0.001319,-0.007050,0.001028,0.008296,0.003162,...,-0.000578,0.008696,0.003181,-0.005956,-0.006820,-0.003631,-0.004041,-0.010083,0.001131,0.003881
2004-12-30,-0.000283,-0.006793,0.005457,-0.001366,-0.005402,-0.002972,-0.002104,0.001027,0.006594,0.005778,...,-0.006940,-0.003592,0.006040,0.000461,-0.004807,0.002024,-0.001449,-0.000764,-0.000226,-0.831716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-12-15,-0.011660,-0.017272,0.004548,-0.007671,-0.004625,0.017857,0.028212,-0.011976,-0.009804,0.003771,...,0.003755,0.115230,0.000426,-0.024162,-0.009021,-0.010531,-0.003533,-0.033080,-0.007280,-0.009213
2005-12-16,-0.022023,-0.028031,-0.017464,-0.011712,0.017472,0.010526,0.026077,-0.011515,-0.009901,-0.006574,...,-0.011224,-0.015460,-0.006242,0.021796,-0.000337,-0.011386,-0.009403,-0.017420,-0.001111,-0.019603
2005-12-19,0.032652,0.009155,0.001317,-0.005689,-0.013153,0.003157,-0.000221,-0.009810,-0.004442,-0.001260,...,0.002703,0.002879,0.008565,0.007167,-0.002867,-0.001252,0.011827,0.014055,-0.004449,-0.004614


In [13]:
eigenvalues, eigenvectors = pca(period, n_components=n_factor)

In [14]:
idxs = [df_returns.columns.get_loc(
            period.columns[i]) for i in range(period.shape[1])]

In [15]:
idxs

[3,
 6,
 12,
 13,
 17,
 23,
 29,
 30,
 31,
 33,
 38,
 44,
 48,
 51,
 54,
 56,
 58,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 70,
 71,
 73,
 75,
 77,
 78,
 80,
 81,
 86,
 87,
 88,
 90,
 93,
 94,
 95,
 99,
 101,
 102,
 103,
 104,
 105,
 107,
 111,
 114,
 117,
 119,
 127,
 130,
 133,
 136,
 138,
 143,
 145,
 150,
 151,
 153,
 158,
 159,
 161,
 162,
 165,
 169,
 171,
 174,
 175,
 177,
 178,
 184,
 185,
 190,
 193,
 194,
 196,
 197,
 198,
 202,
 204,
 205,
 210,
 211,
 213,
 214,
 215,
 217,
 218,
 219,
 225,
 226,
 229,
 230,
 233,
 241,
 242,
 245,
 248,
 249,
 250,
 252,
 254,
 260,
 262,
 265,
 269,
 270,
 275,
 276,
 285,
 292,
 293,
 295,
 297,
 298,
 299,
 301,
 306,
 309,
 317,
 318,
 323,
 325,
 327,
 329,
 332,
 343,
 347,
 348,
 350,
 353,
 355,
 362,
 363,
 370,
 372,
 373,
 374,
 375,
 378,
 381,
 382,
 383,
 385,
 386,
 389,
 393,
 394,
 396,
 401,
 403,
 404,
 407,
 408,
 409,
 412,
 418,
 421,
 422,
 424,
 425,
 427,
 428,
 429,
 430,
 434,
 435,
 436,
 439,
 440,
 442,
 443,
 444

In [16]:
Q[i][:, idxs] = money_on_stock(period, eigenvectors)
Q[i, :, 210]

array([ -7.02515587,   6.02903585, -12.66535338,  -9.27281968,
         3.72115447,  -2.116088  ,   3.61146012,   7.72092138,
         3.40611255,  -6.54064682,  -6.22298018,  -4.82389998,
        -1.03100777,   0.32441801,   1.74802615])

In [17]:
idxs_nonnull = []
for idx in range(Q.shape[2]):
    if Q[i, :, idx].any(axis=0) != 0.:
        print(Q[i, :, idx])
        idxs_nonnull.append(idx)
idxs_nonnull == idxs

[-2.08953434  8.50539801  2.77055588  1.38808794 -0.52188376  2.24652378
 -2.36157714 -2.40132841 -0.36655019 -0.21856186  1.4410843  -1.85854923
  0.73971253  0.6266334   0.87253391]
[-2.40723294 -1.71955091  2.51023391 -3.19000694 -4.82764233  5.33168345
  0.78156084 -3.26173026 -0.60303604  5.79560049 -4.82811776  3.45267036
  5.17069345  6.12768693 -1.32467869]
[-7.237102   -5.10972543 -4.69899548 11.72621332 -8.4148291   7.16081262
 -2.11835348  3.8769038   3.10688341  4.26681084  5.99753529 -0.38016807
 -7.16597649 -7.43278792 -9.42266418]
[-4.76206534 -2.3568564   1.49456798 -0.45752595  0.14453136  1.43303647
 -8.88166608 -2.42926932  1.56655368  0.27543101  1.08256705 -2.59179115
  7.17360373  2.81802001 -5.89956556]
[-3.104957   -2.55061781 -5.43583446  2.39747415 -1.06564721  1.78197531
  3.34207964  0.6179159  -2.21976183  1.28924942 -2.11148516 -0.32922462
 -7.02555317  0.51846937  2.32464603]
[-3.19806134 -2.2900521   1.16257272 -6.18970661 -2.9044471  -1.18544815
 -0.340

True

In [18]:
num = int(np.random.uniform(0,len(idxs)))
if num in idxs:
    print(f'{num} Presente!')
else:
    print(f'{num} Assente!')
Q[i,:,num]

259 Assente!


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
factors = risk_factors(period, Q[i][:, idxs], eigenvectors)

In [20]:
beta_tensor = np.zeros(shape=(trading_days, n_stocks, n_factor))
lookback_for_residual = 60
for stock in period.columns:
        stock_idx = df_returns.columns.get_loc(stock)
        beta0, betas, conf_inter, residuals, pred, _ = regression(
            factors[-lookback_for_residual:], period[-lookback_for_residual:][stock])  # regression(exog, endog) ovvero regression(X,Y)

        beta_tensor[i, stock_idx, :] = betas

        X = np.cumsum(residuals)

In [21]:
num = int(np.random.uniform(0,229))
if num in idxs:
    print('Presente!')
else:
    print('Assente!')
beta_tensor[i, num, :]

Assente!


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
idxs_b = []
for idx in range(beta_tensor.shape[1]):
    if beta_tensor[i, idx, :].any(axis=0) != 0:
        idxs_b.append(idx)

In [23]:
idxs_b == idxs

True

In [27]:
period[stock][-lookback_for_residual:]

Date
2005-09-28   -0.013476
2005-09-29    0.023196
2005-09-30    0.025693
2005-10-03    0.001473
2005-10-04   -0.006376
2005-10-05   -0.047384
2005-10-06    0.006477
2005-10-07   -0.000515
2005-10-10   -0.011074
2005-10-11   -0.010156
2005-10-12   -0.013944
2005-10-13    0.001601
2005-10-14    0.007725
2005-10-17   -0.004230
2005-10-18   -0.024954
2005-10-19    0.009529
2005-10-20   -0.021575
2005-10-21    0.036935
2005-10-24   -0.008240
2005-10-25    0.001340
2005-10-26   -0.012848
2005-10-27    0.008677
2005-10-28    0.034946
2005-10-31   -0.025455
2005-11-01    0.012793
2005-11-02    0.019211
2005-11-03   -0.016525
2005-11-04    0.028091
2005-11-07   -0.003830
2005-11-08    0.008716
2005-11-09    0.003558
2005-11-10    0.015700
2005-11-11    0.002992
2005-11-14   -0.005469
2005-11-15    0.007748
2005-11-16    0.005208
2005-11-17    0.011843
2005-11-18   -0.005121
2005-11-21    0.019853
2005-11-22    0.006729
2005-11-23    0.001671
2005-11-25    0.002383
2005-11-28   -0.013552
2005-1